In [ ]:
#Importing library
import keras
import tensorflow as tf
import numpy as np

##1.Create AlexNet model

###1.1.Create train/test dataset from CIFAR10 dataset

In [ ]:
from keras.datasets import cifar10

In [ ]:
np.random.seed(1)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Đưa các giá trị pixel về dạng số thực
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Đưa miền giá trị về (0,1)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Xử lý tập nhãn
y_train = y_train[:,0]
y_test = y_test[:,0]

cifar10_classes  = ["airplane", "automobile", "bird", "cat", "deer",
                    "dog", "frog", "horse", "ship", "truck",]
y_train = tf.one_hot(y_train, len(cifar10_classes))
y_test = tf.one_hot(y_test, len(cifar10_classes))

In [ ]:
print("Thông tin kích thước của dữ liệu huấn luyện và dữ liệu kiểm tra: ")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

Thông tin kích thước của dữ liệu huấn luyện và dữ liệu kiểm tra: 
(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


###1.2.Create AlexNet model
Nguồn: https://analyticsindiamag.com/hands-on-guide-to-implementing-alexnet-with-keras-for-multi-class-image-classification/

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(2048))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 8, 8, 96)          34944     
                                                                 
 batch_normalization_45 (Bat  (None, 8, 8, 96)         384       
 chNormalization)                                                
                                                                 
 activation_45 (Activation)  (None, 8, 8, 96)          0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 4, 4, 96)         0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 4, 4, 256)         614656    
                                                                 
 batch_normalization_46 (Bat  (None, 4, 4, 256)      

###1.3.Training AlexNet model with CIFAR10 dataset

In [ ]:
def train(model, x_train, y_train): # x_train chinh la X, y_train chinh la ground-truth
  opt = tf.keras.optimizers.SGD(learning_rate=1e-2) # Stochastic Gradient Descent
  model.compile(optimizer=opt, loss='categorical_crossentropy') # CCE loss
  model.fit(x_train, y_train, epochs = 1, batch_size = 64)

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=1e-2) # Stochastic Gradient Descent
AlexNet.compile(optimizer=opt, loss='categorical_crossentropy') # CCE loss
AlexNet.fit(x_train, y_train, epochs = 1, batch_size = 64)

782/782 [==============================] - 1144s 1s/step - loss: 1.6935


##2.Get feature of img in custom dataset

###2.1. Create model to feature extracting with output dim = 2048

In [ ]:
model = Sequential()

for layer in AlexNet.layers[:-4]:
  layer.trainable = False
  model.add(layer)

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 8, 8, 96)          34944     
                                                                 
 batch_normalization_45 (Bat  (None, 8, 8, 96)         384       
 chNormalization)                                                
                                                                 
 activation_45 (Activation)  (None, 8, 8, 96)          0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 4, 4, 96)         0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 4, 4, 256)         614656    
                                                                 
 batch_normalization_46 (Bat  (None, 4, 4, 256)      

In [ ]:
model.get_weights()

[array([[[[ 2.58099753e-02, -4.92358161e-03, -2.23848093e-02, ...,
           -1.00131836e-02,  1.14725558e-02,  5.83317364e-03],
          [ 1.80877559e-02, -1.35314474e-02,  1.80272793e-03, ...,
            2.33315136e-02, -7.89532717e-03,  7.17991358e-03],
          [ 9.70342569e-03, -1.79022253e-02, -2.32566092e-02, ...,
           -1.02155460e-02, -8.34866520e-03,  1.05094733e-02]],
 
         [[ 2.56598480e-02,  1.25147197e-02, -5.08413091e-03, ...,
            1.78949013e-02,  1.52401126e-03, -1.87259763e-02],
          [ 1.50539726e-02,  3.16480151e-03, -2.21525226e-02, ...,
           -9.86011419e-03,  9.02982522e-03,  1.44498236e-03],
          [-1.40165128e-02,  1.70324780e-02, -3.25256735e-02, ...,
           -9.90165304e-03, -2.10206974e-02, -1.26957512e-02]],
 
         [[ 2.10538134e-02,  1.68506429e-02, -7.47277867e-03, ...,
           -1.93028599e-02, -2.94683641e-03, -1.90234371e-02],
          [-4.45922557e-03, -1.56292953e-02,  7.75515521e-03, ...,
            2.247

###2.2.Import Hand/Foot dataset

In [ ]:
import cv2
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
feet_path = "/content/gdrive/MyDrive/Dataset_ARM_FEET/200 feets"
hand_path = "/content/gdrive/MyDrive/Dataset_ARM_FEET/200 hands"

In [ ]:
feet_imgs = [cv2.imread(file) for file in glob.glob(feet_path + "/*.jpg")]
hand_imgs = [cv2.imread(file) for file in glob.glob(hand_path + "/*.jpg")]

###2.3.Get feature of img by model

In [ ]:
feet_imgsResize = [cv2.resize(img, (32, 32)) for img in feet_imgs]
hand_imgsResize = [cv2.resize(img, (32, 32)) for img in hand_imgs]

In [ ]:
feet_features = [model.predict(img.reshape(1,32,32,3)) for img in feet_imgsResize]
hand_features = [model.predict(img.reshape(1,32,32,3)) for img in hand_imgsResize]

1/1 [==============================] - 0s 53ms/step


In [ ]:
feet_features[0].shape

(1, 2048)

##3.Training and Testing feature-dataset with SVM model

###3.1.Train/Test Dataset for train SVM

In [ ]:
X_train = [f.reshape(2048) for f in feet_features[:len(feet_features)//10*7] + hand_features[:len(hand_features)//10*7]]
X_test = [f.reshape(2048) for f in feet_features[len(feet_features)//10*7:] + hand_features[len(hand_features)//10*7:]]
Y_train = [0]*(len(feet_features)//10*7) + [1]*(len(hand_features)//10*7)
Y_test = [0]*(len(feet_features) - len(feet_features)//10*7) + [1]*(len(hand_features) - len(hand_features)//10*7)

###3.2.Create SVM model (1st Time)

In [ ]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

####3.2.1.kernel = 'linear'

In [ ]:
clf = svm.SVC(kernel="linear")
clf.fit(X_train, Y_train)

SVC(kernel='linear')

In [ ]:
Y_predict = clf.predict(X_test)

**Accuracy**

In [ ]:
print('Acc (kernel = "linear"):', accuracy_score(Y_test, Y_predict))

Acc (kernel = "linear"): 0.7416666666666667


**Confusion matrix**

In [ ]:
confusion_matrix(Y_test, Y_predict)

array([[46, 14],
       [17, 43]])

####3.2.2 kernel = 'rbf'

In [ ]:
clf = svm.SVC(kernel="rbf")
clf.fit(X_train, Y_train)

SVC()

In [ ]:
Y_predict = clf.predict(X_test)

**Accuracy**

In [ ]:
print('Acc (kernel = "rbf"):', accuracy_score(Y_test, Y_predict))

Acc (kernel = "rbf"): 0.45


**Confusion matrix**

In [ ]:
confusion_matrix(Y_test, Y_predict)

array([[23, 37],
       [29, 31]])